### Tratamento de dados

In [143]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

df_dados_violencia = pd.read_csv("/content/SINAN-VIOL-2017-2019.csv", low_memory=False)
df_dados_violencia.shape

(1063056, 161)

In [144]:
df_dados_violencia = df_dados_violencia[['level_0', 'level_1','DT_NOTIFIC','DT_OCOR', 'DT_NASC', 'NU_IDADE_N', 'CS_SEXO',
                                         'CS_RACA', 'CS_ESCOL_N', 'LOCAL_OCOR', 'LES_AUTOP', 'VIOL_FISIC', 'VIOL_PSICO',
                                         'VIOL_TORT', 'VIOL_SEXU', 'SEX_ASSEDI', 'SEX_ESTUPR', 'SEX_EXPLO', 'SEX_PORNO',
                                         'SEX_OUTRO', 'REL_PAI', 'REL_MAE', 'REL_PAD', 'REL_MAD', 'REL_CONJ', 'REL_EXCON',
                                         'REL_NAMO', 'REL_EXNAM', 'REL_FILHO', 'REL_IRMAO', 'REL_CONHEC', 'REL_DESCO',
                                         'AUTOR_SEXO', 'OUT_VEZES']]

df_dados_violencia.rename(columns = {'level_0': 'UF', 'level_1': 'ANO'}, inplace = True)
df_dados_violencia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1063056 entries, 0 to 1063055
Data columns (total 34 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   UF          1063056 non-null  object 
 1   ANO         1063056 non-null  int64  
 2   DT_NOTIFIC  1063056 non-null  object 
 3   DT_OCOR     1063055 non-null  object 
 4   DT_NASC     1046169 non-null  object 
 5   NU_IDADE_N  1062115 non-null  float64
 6   CS_SEXO     1063056 non-null  object 
 7   CS_RACA     1057245 non-null  float64
 8   CS_ESCOL_N  1011764 non-null  float64
 9   LOCAL_OCOR  1062582 non-null  float64
 10  LES_AUTOP   1056324 non-null  float64
 11  VIOL_FISIC  1059036 non-null  float64
 12  VIOL_PSICO  1055716 non-null  float64
 13  VIOL_TORT   1052993 non-null  float64
 14  VIOL_SEXU   1053127 non-null  float64
 15  SEX_ASSEDI  1050337 non-null  float64
 16  SEX_ESTUPR  1051530 non-null  float64
 17  SEX_EXPLO   1049729 non-null  float64
 18  SEX_PORNO   1049791 no

In [145]:
df_dados_violencia = df_dados_violencia[df_dados_violencia['CS_SEXO']=='F']
df_dados_violencia = df_dados_violencia[df_dados_violencia['LES_AUTOP']!=1]
df_dados_violencia.drop(['CS_SEXO', 'LES_AUTOP', 'DT_NOTIFIC'], axis = 1, inplace = True)
df_dados_violencia.dropna(subset=['DT_OCOR'], inplace = True)

df_dados_violencia['UF'] = df_dados_violencia['UF'].astype('category')
df_dados_violencia['ANO'] = df_dados_violencia['ANO'].astype('category')
df_dados_violencia['REL_PAI'] = pd.to_numeric(df_dados_violencia['REL_PAI'], errors='coerce')
df_dados_violencia['DT_NASC'] = pd.to_datetime(df_dados_violencia['DT_NASC'])
df_dados_violencia['DT_OCOR'] = pd.to_datetime(df_dados_violencia['DT_OCOR'])
df_dados_violencia.info()

<class 'pandas.core.frame.DataFrame'>
Index: 564585 entries, 0 to 1063054
Data columns (total 31 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   UF          564585 non-null  category      
 1   ANO         564585 non-null  category      
 2   DT_OCOR     564585 non-null  datetime64[ns]
 3   DT_NASC     555844 non-null  datetime64[ns]
 4   NU_IDADE_N  564239 non-null  float64       
 5   CS_RACA     561800 non-null  float64       
 6   CS_ESCOL_N  537901 non-null  float64       
 7   LOCAL_OCOR  564307 non-null  float64       
 8   VIOL_FISIC  562891 non-null  float64       
 9   VIOL_PSICO  561490 non-null  float64       
 10  VIOL_TORT   559870 non-null  float64       
 11  VIOL_SEXU   559997 non-null  float64       
 12  SEX_ASSEDI  557540 non-null  float64       
 13  SEX_ESTUPR  558645 non-null  float64       
 14  SEX_EXPLO   557051 non-null  float64       
 15  SEX_PORNO   557100 non-null  float64       
 16  SEX_OU

In [146]:
# Registros sem idade e sem data de nasc.
sidade_snasc = df_dados_violencia[df_dados_violencia['NU_IDADE_N'].isna() & df_dados_violencia['DT_NASC'].isna()]

print(f'Há {len(sidade_snasc)} registros sem idade e sem a data de nascimento.')

Há 153 registros sem idade e sem a data de nascimento.


In [147]:
# Excluir os registros sem idade e sem data de nascimento
df_dados_violencia = df_dados_violencia[(df_dados_violencia['NU_IDADE_N'].notnull() |
                                         df_dados_violencia['DT_NASC'].notnull())]

In [148]:
# Colocar idade em anos
def calcula_idade(idade, nasc, ocor):
    if idade != idade: # Retorna True caso idade = nan
        return ocor.year - nasc.year - ((ocor.month, ocor.day) < (nasc.month, nasc.day))
    elif (idade > 4000):
        return idade - 4000
    elif (idade > 3000):
        return (idade - 3000)/12
    elif (idade > 1000):
        return 0
    else:
        return idade

df_dados_violencia['NU_IDADE_N'] = df_dados_violencia.apply(lambda row: calcula_idade(row['NU_IDADE_N'],
                                                                                      row['DT_NASC'],
                                                                                      row['DT_OCOR']), axis=1)

df_dados_violencia['NU_IDADE_N'].isnull().sum()

np.int64(0)

In [149]:


# Criar campo GRUPO_IDADE para agrupar idades e ter a visualização dos dados por faixa etária:
bins= [0, 2, 6, 10, 14, 18, 22, 26, 30, 34, 38, 42, 46, 50, 55, 60, 66, 120]
labels = ['< de 2 anos','2 a 5 anos','6 a 9 anos','10 a 13 anos','14 a 17 anos','18 a 21 anos','22 a 25 anos',
          '26 a 29 anos', '30 a 33 anos','34 a 37 anos','38 a 41 anos','42 a 45 anos','46 a 49 anos','50 a 54 anos',
          '55 a 59 anos','60 a 65 anos','> de 65 anos']

df_dados_violencia['GRUPO_IDADE'] = pd.cut(df_dados_violencia['NU_IDADE_N'], bins=bins, labels=labels,
                                           right=False).astype("category")
df_dados_violencia



,UF,ANO,DT_OCOR,DT_NASC,NU_IDADE_N,CS_RACA,CS_ESCOL_N,LOCAL_OCOR,VIOL_FISIC,VIOL_PSICO,...,REL_EXCON,REL_NAMO,REL_EXNAM,REL_FILHO,REL_IRMAO,REL_CONHEC,REL_DESCO,AUTOR_SEXO,OUT_VEZES,GRUPO_IDADE
0,AC,2017,2016-06-15,2002-05-03,14.0,4.0,3.0,1.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,14 a 17 anos
1,AC,2017,2016-12-31,1975-10-29,41.0,4.0,5.0,5.0,1.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,38 a 41 anos
3,AC,2017,2016-12-31,1988-04-01,28.0,4.0,4.0,6.0,1.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,26 a 29 anos
5,AC,2017,2017-01-01,1985-06-12,31.0,4.0,2.0,1.0,2.0,2.0,...,1.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,30 a 33 anos
6,AC,2017,2017-01-02,1999-12-23,17.0,4.0,3.0,1.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,14 a 17 anos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1063047,TO,2019,2019-03-25,2004-11-04,14.0,4.0,3.0,1.0,1.0,2.0,...,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0,14 a 17 anos
1063049,TO,2019,2019-03-23,1999-07-07,19.0,4.0,9.0,1.0,1.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,9.0,18 a 21 anos
1063052,TO,2019,2018-12-22,2016-07-17,2.0,4.0,10.0,1.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,9.0,2 a 5 anos
1063053,TO,2019,2019-01-08,1989-03-19,29.0,4.0,3.0,1.0,1.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,26 a 29 anos


In [150]:
df_dados_violencia['CS_RACA'] = df_dados_violencia['CS_RACA'].astype('category')

mapping = {1 : 'Branca',
           2 : 'Preta',
           3 : 'Amarela',
           4 : 'Parda',
           5 : 'Indígena',
           9 : 'Ignorado'}

df_dados_violencia['CS_RACA'] = df_dados_violencia['CS_RACA'].map(mapping)
df_dados_violencia['CS_RACA'].value_counts()

,count
CS_RACA,
Parda,235939
Branca,216549
Ignorado,50694
Preta,49199
Indígena,5129
Amarela,4138


In [151]:
df_dados_violencia['CS_ESCOL_N'] = df_dados_violencia['CS_ESCOL_N'].astype('category')

mapping = {0 : 'Não alfabetizada',
           1 : '1ª a 4ª série incompleta do EF',
           2 : '4ª série completa do EF',
           3 : '5ª à 8ª série incompleta do EF',
           4 : 'Ensino fundamental completo',
           5 : 'Ensino médio incompleto',
           6 : 'Ensino médio completo',
           7 : 'Educação superior incompleta',
           8 : 'Educação superior completa',
           9 : 'Ignorado ou em branco',
           10 : 'Não se aplica'}

df_dados_violencia['CS_ESCOL_N'] = df_dados_violencia['CS_ESCOL_N'].map(mapping)
df_dados_violencia['CS_ESCOL_N'].value_counts()

,count
CS_ESCOL_N,
Ignorado ou em branco,143307
5ª à 8ª série incompleta do EF,79793
Ensino médio completo,76710
Não se aplica,62734
Ensino médio incompleto,49236
Ensino fundamental completo,36081
1ª a 4ª série incompleta do EF,35681
4ª série completa do EF,18183
Educação superior completa,15610


In [152]:
df_dados_violencia['LOCAL_OCOR'] = df_dados_violencia['LOCAL_OCOR'].astype('category')

mapping = {1 : 'Residência',
           2 : 'Habitação coletiva',
           3 : 'Escola',
           4 : 'Local de prática esportiva',
           5 : 'Bar ou similar',
           6 : 'Via publica',
           7 : 'Comércio/Serviços',
           8 : 'Industrias/ construção',
           9 : 'Outro'}

df_dados_violencia['LOCAL_OCOR'] = df_dados_violencia['LOCAL_OCOR'].map(mapping)
df_dados_violencia['LOCAL_OCOR'].value_counts()

,count
LOCAL_OCOR,
Residência,352502
Via publica,75492
Outro,33040
Bar ou similar,11541
Comércio/Serviços,9504
Escola,8957
Habitação coletiva,4412
Local de prática esportiva,1303
Industrias/ construção,606


In [153]:
df_dados_violencia.dtypes

,0
UF,category
ANO,category
DT_OCOR,datetime64[ns]
DT_NASC,datetime64[ns]
NU_IDADE_N,float64
CS_RACA,category
CS_ESCOL_N,category
LOCAL_OCOR,object
VIOL_FISIC,float64
VIOL_PSICO,float64


In [154]:
# Cria uma categoria única simplificada por linha
def categorizar_violencia_simplificada(row):
    tipos = []
    if row['VIOL_FISIC'] == 1:
        tipos.append('física')
    if row['VIOL_PSICO'] == 1:
        tipos.append('psicológica')
    if row['VIOL_SEXU'] == 1 or row['SEX_ASSEDI'] == 1 or row['SEX_ESTUPR'] == 1 or row['SEX_EXPLO'] == 1 or row['SEX_PORNO'] == 1 or row['SEX_OUTRO'] == 1:
        tipos.append('sexual')
    if row['VIOL_TORT'] == 1:
        tipos.append('tortura')

    if not tipos:
        return 'nenhuma'
    elif len(tipos) == 1:
        return tipos[0]
    else:
        return 'múltiplas'

df_dados_violencia['TIPO_VIOLENCIA'] = df_dados_violencia.apply(categorizar_violencia_simplificada, axis=1)

In [155]:
colunas_violencia = [
    'VIOL_FISIC', 'VIOL_PSICO', 'VIOL_TORT', 'VIOL_SEXU',
    'SEX_ASSEDI', 'SEX_ESTUPR', 'SEX_EXPLO', 'SEX_PORNO', 'SEX_OUTRO'
]

df_dados_violencia.drop(columns=colunas_violencia, inplace=True)


In [156]:
def categorizar_relacao(row):
    grupos = set()

    # Família
    if row['REL_PAI'] == 1 or row['REL_MAE'] == 1 or row['REL_PAD'] == 1 or row['REL_MAD'] == 1 or \
       row['REL_FILHO'] == 1 or row['REL_IRMAO'] == 1:
        grupos.add('familiar')

    # Cônjuge e ex-cônjuge
    if row['REL_CONJ'] == 1 or row['REL_EXCON'] == 1:
        grupos.add('cônjuge')

    # Namorado(a) e ex
    if row['REL_NAMO'] == 1 or row['REL_EXNAM'] == 1:
        grupos.add('namorado(a)')

    # Conhecido
    if row['REL_CONHEC'] == 1:
        grupos.add('conhecido(a)')

    # Desconhecido
    if row['REL_DESCO'] == 1:
        grupos.add('desconhecido(a)')

    if not grupos:
        return 'não informado'
    elif len(grupos) == 1:
        return list(grupos)[0]
    else:
        return 'múltiplas'

df_dados_violencia['TIPO_RELACAO'] = df_dados_violencia.apply(categorizar_relacao, axis=1)


In [157]:
colunas_relacao = [
    'REL_PAI', 'REL_MAE', 'REL_PAD', 'REL_MAD',
    'REL_CONJ', 'REL_EXCON', 'REL_NAMO', 'REL_EXNAM',
    'REL_FILHO', 'REL_IRMAO', 'REL_CONHEC', 'REL_DESCO'
]

df_dados_violencia.drop(columns=colunas_relacao, inplace=True)


In [158]:
df_dados_violencia.info()

<class 'pandas.core.frame.DataFrame'>
Index: 564432 entries, 0 to 1063054
Data columns (total 13 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   UF              564432 non-null  category      
 1   ANO             564432 non-null  category      
 2   DT_OCOR         564432 non-null  datetime64[ns]
 3   DT_NASC         555844 non-null  datetime64[ns]
 4   NU_IDADE_N      564432 non-null  float64       
 5   CS_RACA         561648 non-null  category      
 6   CS_ESCOL_N      537748 non-null  category      
 7   LOCAL_OCOR      497357 non-null  object        
 8   AUTOR_SEXO      562987 non-null  float64       
 9   OUT_VEZES       560070 non-null  float64       
 10  GRUPO_IDADE     564416 non-null  category      
 11  TIPO_VIOLENCIA  564432 non-null  object        
 12  TIPO_RELACAO    564432 non-null  object        
dtypes: category(5), datetime64[ns](2), float64(3), object(3)
memory usage: 41.5+ MB


In [159]:
df_dados_violencia['TIPO_VIOLENCIA'] = df_dados_violencia['TIPO_VIOLENCIA'].astype('category')
df_dados_violencia['TIPO_RELACAO'] = df_dados_violencia['TIPO_RELACAO'].astype('category')
df_dados_violencia['LOCAL_OCOR'] = df_dados_violencia['LOCAL_OCOR'].astype('category')

In [160]:
df_dados_violencia.dtypes

,0
UF,category
ANO,category
DT_OCOR,datetime64[ns]
DT_NASC,datetime64[ns]
NU_IDADE_N,float64
CS_RACA,category
CS_ESCOL_N,category
LOCAL_OCOR,category
AUTOR_SEXO,float64
OUT_VEZES,float64


In [161]:
df_dados_violencia.isnull().sum()

,0
UF,0
ANO,0
DT_OCOR,0
DT_NASC,8588
NU_IDADE_N,0
CS_RACA,2784
CS_ESCOL_N,26684
LOCAL_OCOR,67075
AUTOR_SEXO,1445
OUT_VEZES,4362


In [162]:
# Preenchendo colunas categóricas com a moda
colunas_moda = ["CS_RACA", "CS_ESCOL_N", "LOCAL_OCOR", "GRUPO_IDADE"]
for coluna in colunas_moda:
    moda = df_dados_violencia[coluna].mode()[0]
    df_dados_violencia[coluna].fillna(moda, inplace=True)

# Preenchendo colunas numéricas com a mediana
colunas_mediana = ["AUTOR_SEXO", "OUT_VEZES"]
for coluna in colunas_mediana:
    mediana = df_dados_violencia[coluna].median()
    df_dados_violencia[coluna].fillna(mediana, inplace=True)

# Conferindo resultado
print(df_dados_violencia.isnull().sum())


UF                   0
ANO                  0
DT_OCOR              0
DT_NASC           8588
NU_IDADE_N           0
CS_RACA              0
CS_ESCOL_N           0
LOCAL_OCOR           0
AUTOR_SEXO           0
OUT_VEZES            0
GRUPO_IDADE          0
TIPO_VIOLENCIA       0
TIPO_RELACAO         0
dtype: int64


<ipython-input-162-fdd2875da82e>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_dados_violencia[coluna].fillna(moda, inplace=True)
<ipython-input-162-fdd2875da82e>:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', 

In [163]:
df_dados_violencia.drop(columns=['DT_NASC'], inplace=True)

In [164]:
df_dados_violencia.isnull().sum()

,0
UF,0
ANO,0
DT_OCOR,0
NU_IDADE_N,0
CS_RACA,0
CS_ESCOL_N,0
LOCAL_OCOR,0
AUTOR_SEXO,0
OUT_VEZES,0
GRUPO_IDADE,0


In [165]:
df_dados_violencia['DT_OCOR'].head()

,DT_OCOR
0,2016-06-15
1,2016-12-31
3,2016-12-31
5,2017-01-01
6,2017-01-02


In [166]:
df_dados_violencia.drop(columns=['DT_OCOR'], inplace=True)

### Aplicação de OneHot/Label + algoritmo

In [167]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Lista de colunas para OneHotEncoder (sem 'TIPO_VIOLENCIA')
colunas_onehot = [
    'UF',
    'CS_RACA',
    'CS_ESCOL_N',
    'LOCAL_OCOR',
    'GRUPO_IDADE',
    'TIPO_RELACAO'
]

# LabelEncoder para a coluna ANO (se quiser manter como ordinal)
le = LabelEncoder()
df_dados_violencia['ANO'] = le.fit_transform(df_dados_violencia['ANO'])

# OneHot Encoding diretamente no DataFrame original
# df_dados_violencia = pd.get_dummies(df_dados_violencia, columns=colunas_onehot, drop_first=True)

# Conferindo o resultado
print(df_dados_violencia.head())


   UF  ANO  NU_IDADE_N CS_RACA                      CS_ESCOL_N  \
0  AC    0        14.0   Parda  5ª à 8ª série incompleta do EF   
1  AC    0        41.0   Parda         Ensino médio incompleto   
3  AC    0        28.0   Parda     Ensino fundamental completo   
5  AC    0        31.0   Parda         4ª série completa do EF   
6  AC    0        17.0   Parda  5ª à 8ª série incompleta do EF   

       LOCAL_OCOR  AUTOR_SEXO  OUT_VEZES   GRUPO_IDADE TIPO_VIOLENCIA  \
0      Residência         1.0        1.0  14 a 17 anos         sexual   
1  Bar ou similar         1.0        2.0  38 a 41 anos         física   
3     Via publica         1.0        2.0  26 a 29 anos         física   
5      Residência         1.0        2.0  30 a 33 anos         sexual   
6      Residência         1.0        1.0  14 a 17 anos         sexual   

      TIPO_RELACAO  
0          cônjuge  
1  desconhecido(a)  
3  desconhecido(a)  
5          cônjuge  
6          cônjuge  


In [168]:
df_dados_violencia.columns

Index(['UF', 'ANO', 'NU_IDADE_N', 'CS_RACA', 'CS_ESCOL_N', 'LOCAL_OCOR',
       'AUTOR_SEXO', 'OUT_VEZES', 'GRUPO_IDADE', 'TIPO_VIOLENCIA',
       'TIPO_RELACAO'],
      dtype='object')

In [169]:
# 1. Separar o alvo ANTES do get_dummies
y = df_dados_violencia['TIPO_VIOLENCIA']

# 2. Remover a coluna do DataFrame
X = df_dados_violencia.drop(columns=['TIPO_VIOLENCIA'])

# 3. Aplicar get_dummies só nas colunas categóricas dos preditores
colunas_onehot = [
    'UF', 'CS_RACA', 'CS_ESCOL_N', 'LOCAL_OCOR',
    'GRUPO_IDADE', 'TIPO_RELACAO'  # repara que 'TIPO_VIOLENCIA' saiu daqui
]

X = pd.get_dummies(X, columns=colunas_onehot, drop_first=True)

# 4. Continuar com o modelo
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

modelo = RandomForestClassifier(random_state=42)
modelo.fit(X_train, y_train)

y_pred = modelo.predict(X_test)

print("Relatório de Classificação:\n")
print(classification_report(y_test, y_pred))


Relatório de Classificação:

              precision    recall  f1-score   support

      física       0.60      0.69      0.65     47501
   múltiplas       0.43      0.37      0.40     31794
     nenhuma       0.74      0.75      0.74     11906
 psicológica       0.27      0.15      0.20      7712
      sexual       0.58      0.59      0.59     13907
     tortura       0.00      0.00      0.00        67

    accuracy                           0.56    112887
   macro avg       0.44      0.43      0.43    112887
weighted avg       0.54      0.56      0.55    112887

